## Create plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import cycler
plt.rcParams.update({'font.size': 14})

# import magnetization and steady state plot create functions
from plot_functions import *

### Folder with output files 

In [ ]:
folder_path = './one_dim_outputs/'
sorted(os.listdir(folder_path))


### Load output csvs for specified method, media values and campaign splits

In [ ]:
method = 'default'
C = 3 # number of ratio splits

# media_list = [0.0, 0.001, 0.003, 0.005, 0.01, 0.03, 0.05, 0.1]
media_list = [0.0, 0.001, 0.005]
m_to_df, camps = load_data(folder_path, method, C, media_list)

### Plot magnetization 
#### Use group = True and mention grouby_idx (the campaign split id) to plot curves for different media values on plot to compare.
#### Pass "save=True" to save figures

In [ ]:
plot_magnetization(m_to_df, camps, method, path = folder_path, group = False, groupby_idx = 0)

### Plot steady states
#### Pass "save=True" to save figures

In [ ]:
plot_ss(m_to_df, camps, method, folder_path)

## Plot Decision times

In [ ]:
# specify parameters    
conc = 0.5
L = 24*24
split = 0


no_weights = dict.fromkeys(media_list)
for m in sorted(media_list):
    no_weights[m] = m_to_df[m]

plt.figure(figsize = (8, 6))
for m in media_list:
# for i in range(3):
    df = no_weights[m][split]
    val_list = []
    num_sims = (max(df.sim)+1)
    df = df.query('conc==%f'%conc)
    for row in df.itertuples():
        [vals, counts] = json.loads(row.decision)
        all_vals = [[vals[i]]*int(counts[i]) for i in range(len(vals))]
        all_vals_open = [int(value/L) for sublist in all_vals for value in sublist]
        val_list.append(all_vals_open)
    val_list = np.array([value for sublist in val_list for value in sublist if value!=0])
    v, counts = np.unique(val_list, return_counts=True)
    
    if(m==0.0):
        plt.loglog((v), counts/sum(counts),'+', label = "p = %0.3f"%m)
        fit = powerlaw.Fit(val_list[np.where(val_list>0)], xmin=min(val_list))
        fit.power_law.plot_pdf(color= 'orange',label='powerfit'+r'$ \approx \mu^{-%0.3f}$'%np.round(fit.power_law.alpha, 3))
        print('alpha= ',fit.power_law.alpha,'  sigma= ',fit.power_law.sigma)
    else:
#         plt.loglog((v), counts/sum(counts),'+--', label = "m="+str(camps[i].split('_')[0]))#label = "p = %0.3f"%m)
        plt.loglog((v), counts/sum(counts),'+--', label = "p = %0.3f"%m)
    
plt.legend(loc = 0, fontsize = 14)
plt.title("Decision time distribution", fontsize = 14)
plt.xlabel('log of decision times '+r'($\mu$)', fontsize = 14)
plt.ylabel('log of probability', fontsize = 14)
plt.grid(True)
# plt.savefig(folder_path+"_"+str(m)+"_"+str(2)+"_dectimes.png", dpi = 100)


## Plot Relaxation Times

In [ ]:
for m in media_list:    
    num_sims = (max(m_to_df[m][0].sim)+1)
    plt.figure(figsize=(8,6))
    concs = np.unique(m_to_df[m][0].conc)
    conc_rel = np.zeros((len(concs), len(m_to_df[m].keys())))
    for split in sorted(m_to_df[m]):

        print(camps[split])
        df = m_to_df[m][split]
        df = df[['conc','relaxation']].groupby('conc').sum()/(num_sims*24*24)
        plt.plot(df, label = camps[split].split('_')[0])
        conc_rel[:, split] = (np.array(df.relaxation))

    plt.xlabel("intital upward spin concentration "+r"$C_u$", fontsize = 14)
    plt.ylabel("relaxation time", fontsize = 14)
    plt.title(r"Relaxation times for $p = %.3f$"%m, fontsize = 14)
    plt.legend(loc=0)
    plt.grid(True)

#     plt.savefig(folder_path+"RelaxationTime_comp_"+method+".png")
    plt.show()